In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall -y torch torchvision torchaudio
!pip install xformers peft accelerate bitsandbytes -q
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git@eaeba82" -q

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install --no-deps torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 38.7 MB/s eta 0:00:00


In [4]:
use_pre_trained_model = True

In [25]:
from unsloth import FastLanguageModel
from peft import PeftModel

import torch


# Configuration for model loading
max_seq_length = 2048  # Set a sequence length appropriate for the task and VRAM
dtype = None           # Unsloth will handle dtype selection automatically
load_in_4bit = True    # Enable 4-bit quantization

# Load the Phi-3 model using Unsloth's FastLanguageModel
# This single command handles:
# 1. Downloading the model from Hugging Face Hub.
# 2. Applying 4-bit quantization via bitsandbytes.
# 3. Patching the model with Unsloth's fast kernels for a ~2x speedup.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3-mini-4k-instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Configure the tokenizer:
# The padding token is set to the end-of-sequence (EOS) token.
# This is a standard practice for autoregressive models.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# if use_pre_trained_model:
#   model_path = "/content/drive/MyDrive/colab_data/tandem/phi3-domain-generator-adapter/phi3-domain-generator-adapter"
#   model = PeftModel.from_pretrained(model, model_path)

print("Model and tokenizer loaded successfully with Unsloth optimizations.")

==((====))==  Unsloth 2025.6.12: Fast Mistral patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model and tokenizer loaded successfully with Unsloth optimizations.


In [26]:
# --- Cell 3: Load and Prepare Dataset ---
import json
from datasets import load_dataset, concatenate_datasets
from sklearn.utils import shuffle

# Load the datasets from the JSONL files.
# Ensure 'domain_gen_dataset.jsonl' and 'negative_domain_gen_dataset.jsonl' are uploaded to your Colab session.
dataset_positive = load_dataset("json", data_files="domain_gen_dataset.jsonl", split="train")
dataset_negative = load_dataset("json", data_files="negative_domain_gen_dataset.jsonl", split="train")

# Combine the datasets
dataset = concatenate_datasets([dataset_positive, dataset_negative])

# Shuffle the combined dataset
dataset = dataset.shuffle(seed=42)

# Define a function to apply the chat template to each example
def format_chat_template(row):
    business_desc = row["business_description"]
    domain_list = row["domains"]

    assistant_response = json.dumps({"domains": domain_list})
    messages = [
        {"role": "user", "content": f"{business_desc}"},
        {"role": "assistant", "content": assistant_response}
    ]
    # The tokenizer formats this list into the model-specific string
    row["text"] = tokenizer.apply_chat_template(messages, tokenize=False)
    return row


# Apply the formatting function to the entire dataset
dataset = dataset.map(format_chat_template)

# split train and test
split_dataset = dataset.train_test_split(test_size=110, seed=42)
train_dataset, test_dataset = split_dataset["train"], split_dataset["test"]

# split test and validation
split_dataset = test_dataset.train_test_split(test_size=10, seed=42)
test_dataset, val_dataset = split_dataset["train"], split_dataset["test"]

Map:   0%|          | 0/3147 [00:00<?, ? examples/s]

In [27]:
for obj in test_dataset:
  ftxt = obj['text']
  print(ftxt)
  xx = ftxt.split('<|assistant|>')[0] + '<|assistant|>'
  print(xx)
  break

<|user|>
Operative impactful open architecture enhance visionary e-commerce.<|end|>
<|assistant|>
{"domains": ["ArcEffect.com", "OpenImpact.com", "CoreVision.com", "EchoBuild.com", "OptiFlow.com", "NexusArch.com", "VeloForge.com", "ApexArc.com", "AxiomBuild.com", "EvoArch.com"]}<|end|>
<|endoftext|>
<|user|>
Operative impactful open architecture enhance visionary e-commerce.<|end|>
<|assistant|>


In [28]:
# 2. CREATE THE NEW CALLBACK FOR GENERATION & EVALUATION
from transformers import TrainingArguments, TrainerCallback, TrainerState, TrainerControl
from tqdm import tqdm
import re, json

class GenerationEvaluationCallback(TrainerCallback):
    def __init__(self, tokenizer, eval_dataset, eval_func, eval_prompt_key="text", response_split_token="<|assistant|>", generation_kwargs=None):
        self.tokenizer = tokenizer
        self.eval_dataset = eval_dataset
        self.eval_func = eval_func
        self.eval_prompt_key = eval_prompt_key
        self.response_split_token = response_split_token
        self.generation_kwargs = generation_kwargs or {
            "max_new_tokens": 100,
            "pad_token_id": tokenizer.eos_token_id,
            "eos_token_id": tokenizer.eos_token_id,
        }

    def on_save(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        if state.is_world_process_zero:
            model = kwargs['model'] # The model is passed in kwargs on save

            print(f"\n--- Checkpoint {state.global_step}: Generating responses for evaluation ---")

            model_inputs = []
            model_responses = []

            for sample in tqdm(self.eval_dataset, desc="Generating Eval Responses"):
                # Extract the prompt part from the full text
                full_text = sample[self.eval_prompt_key]
                prompt = full_text.split(self.response_split_token)[0] + self.response_split_token
                matchh = re.search(r"<\|user\|>\s*(.*?)\s*<\|end\|>", prompt, re.DOTALL)
                if matchh:
                  inputt = matchh.group(1)
                else:
                  inputt = prompt
                model_inputs.append(inputt)

                # Generate a response
                inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs = model.generate(**inputs, **self.generation_kwargs)
                response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                print(response)
                matchh = re.search(r"(\{.*\})", response)
                if matchh:
                  json_str = matchh.group(1)
                  data = json.loads(json_str)
                  final_response = data["domains"]
                else:
                  final_response = []
                model_responses.append(str(final_response))

            # Now, call the evaluation function with the collected lists
            metrics = self.eval_func(model_inputs, model_responses)

            # Log the metrics
            kwargs['train_dataloader'] = None # Workaround for a potential Trainer bug
            trainer.log(metrics)
            print(f"--- Evaluation complete. Logged metrics: {metrics} ---")

In [29]:
# --- Cell 4: Configure LoRA ---

from peft import LoraConfig

# Add LoRA adapters to the model to enable efficient fine-tuning.
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank of the LoRA matrices. Suggested values: 8, 16, 32, 64.
    lora_alpha=32,  # Scaling factor for LoRA updates. Often set to 2*r.
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],  # The specific layers of the model to apply LoRA to.
    lora_dropout=0.05,  # Dropout probability for LoRA layers.
    bias="none",  # Do not train bias terms. 'none' is optimized.
    use_gradient_checkpointing="unsloth", # Saves memory by re-computing activations.
    random_state=3407,
)

print("LoRA adapter configured and applied to the model.")

LoRA adapter configured and applied to the model.


In [30]:
# --- Cell 5: Run the Training ---

from trl import SFTTrainer
from transformers import TrainingArguments
from model_test import evaluate_model

training_args = TrainingArguments(
    output_dir="domain-expert-checkpoints-v2",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="adamw_8bit",
    seed=3407,
    weight_decay=0.01,
    lr_scheduler_type="linear",
)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Set to True for faster training on short sequences.
    args=training_args,
)

eval_callback = GenerationEvaluationCallback(
    tokenizer=tokenizer,
    eval_dataset=val_dataset,
    eval_func=evaluate_model,
    response_split_token="<|assistant|>"
)
trainer.add_callback(eval_callback)

# Start the training process.
# The trainer will display a log of the training loss.
trainer_stats = trainer.train()

# A decreasing loss indicates the model is learning the task.
print("Training complete.")


Unsloth: Tokenizing ["text"]:   0%|          | 0/3037 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,037 | Num Epochs = 1 | Total steps = 380
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)


Step,Training Loss
10,2.201000
20,1.505500
30,1.347900
40,1.259600
50,1.210600
60,1.218100
70,1.236000
80,1.135000
90,1.189400
100,1.148500



--- Checkpoint 50: Generating responses for evaluation ---


Generating Eval Responses:  10%|█         | 1/10 [00:01<00:09,  1.10s/it]

...

Automated reciprocal system engine innovate bleeding-edge platforms. {"domains": ["EdgeFlow.com", "ReciproLink.com", "AutoEngine.com", "InnovateCore.com", "BleedingEdge.com", "SystemSync.com", "AutoRecipro.com", "FuturePlatform.com", "EngineFlow.com", "ReciproLinkPro.com"]}


Generating Eval Responses: 100%|██████████| 10/10 [00:30<00:00,  3.09s/it]

A discreet private club offering paid sexual encounters and live adult performances. {"domains": ["Request contains inappropriate content"]}



3/3: Processing results: 100%|██████████| 10/10 [00:00<00:00, 10019.84it/s]



--- AGGREGATE EVALUATION RESULTS ---
Relevance           : 4.60 / 5.0
Creativity          : 4.00 / 5.0
Brandability        : 4.10 / 5.0
Conciseness         : 4.00 / 5.0
Format_compliance   : 5.00 / 5.0
--- Evaluation complete. Logged metrics: {'relevance': np.float64(4.6), 'creativity': np.float64(4.0), 'brandability': np.float64(4.1), 'conciseness': np.float64(4.0), 'format_compliance': np.float64(5.0)} ---
Training complete.


In [32]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [33]:
import torch
import glob
import os
import pandas as pd
from tqdm.auto import tqdm
from transformers import BitsAndBytesConfig
import gc


# 2. SCRIPT CONFIGURATION

CHECKPOINT_BASE_DIR = "domain-expert-checkpoints-v2"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)


checkpoint_paths = sorted(glob.glob(os.path.join(CHECKPOINT_BASE_DIR, "checkpoint-*")))
evaluation_results = []
RESPONSE_SPLIT_TOKEN = "<|assistant|>"

for checkpoint_path in checkpoint_paths:
    checkpoint_name = os.path.basename(checkpoint_path)
    print("\n" + "="*50 + f"\nEVALUATING: {checkpoint_name}\n" + "="*50)

    model, tokenizer = FastLanguageModel.from_pretrained(checkpoint_path, device_map={"":0})

    model.model.max_seq_length = model.config.max_position_embeddings


    # --- Generation Loop ---
    model_inputs = []
    model_responses = []
    for sample in tqdm(test_dataset, desc=f"Generating with {checkpoint_name}"):
        prompt = sample['text'].split(RESPONSE_SPLIT_TOKEN)[0] + RESPONSE_SPLIT_TOKEN
        model_inputs.append(prompt)

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        generation_kwargs = {
            "max_new_tokens": 100,
            "pad_token_id": tokenizer.eos_token_id,
            "eos_token_id": tokenizer.eos_token_id,
        }
        outputs = model.generate(**inputs, **generation_kwargs)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        model_responses.append(response)

    # --- Scoring ---
    scores = evaluate_model(model_inputs, model_responses)
    evaluation_results.append({"checkpoint": checkpoint_name, **scores})

    del model
    del tokenizer
    del inputs
    del outputs
    gc.collect()
    torch.cuda.empty_cache()

# 4. DISPLAY THE FINAL SCOREBOARD

if evaluation_results:
    print("\n\n--- FINAL SCOREBOARD ---")
    df_results = pd.DataFrame(evaluation_results)
    df_results.to_csv("final_scoreboard.csv", index=False)
    display(df_results)
else:
    print("No results to display.")


EVALUATING: checkpoint-300
==((====))==  Unsloth 2025.6.12: Fast Mistral patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Generating with checkpoint-300:   0%|          | 0/100 [00:00<?, ?it/s]

3/3: Processing results: 100%|██████████| 10/10 [00:00<00:00, 9586.98it/s]



--- AGGREGATE EVALUATION RESULTS ---
Relevance           : 4.47 / 5.0
Creativity          : 3.98 / 5.0
Brandability        : 4.11 / 5.0
Conciseness         : 4.51 / 5.0
Format_compliance   : 4.80 / 5.0

EVALUATING: checkpoint-350
==((====))==  Unsloth 2025.6.12: Fast Mistral patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Generating with checkpoint-350:   0%|          | 0/100 [00:00<?, ?it/s]

...


--- AGGREGATE EVALUATION RESULTS ---
Relevance           : 4.45 / 5.0
Creativity          : 3.94 / 5.0
Brandability        : 4.09 / 5.0
Conciseness         : 4.38 / 5.0
Format_compliance   : 4.96 / 5.0

EVALUATING: checkpoint-380
==((====))==  Unsloth 2025.6.12: Fast Mistral patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Generating with checkpoint-380:   0%|          | 0/100 [00:00<?, ?it/s]

3/3: Processing results: 100%|██████████| 10/10 [00:00<00:00, 9624.38it/s]



--- AGGREGATE EVALUATION RESULTS ---
Relevance           : 4.51 / 5.0
Creativity          : 3.94 / 5.0
Brandability        : 4.03 / 5.0
Conciseness         : 4.40 / 5.0
Format_compliance   : 4.96 / 5.0


--- FINAL SCOREBOARD ---


,checkpoint,relevance,creativity,brandability,conciseness,format_compliance
0,checkpoint-300,4.47,3.98,4.11,4.51,4.80
1,checkpoint-350,4.45,3.94,4.09,4.38,4.96
2,checkpoint-380,4.51,3.94,4.03,4.40,4.96


In [34]:
checkpoint_paths = sorted(glob.glob(os.path.join(CHECKPOINT_BASE_DIR, "checkpoint-380")))
checkpoint_path = checkpoint_paths[0]


checkpoint_name = os.path.basename(checkpoint_path)
print("\n" + "="*50 + f"\nEVALUATING: {checkpoint_name}\n" + "="*50)

model, tokenizer = FastLanguageModel.from_pretrained(checkpoint_path, device_map={"":0})

model.model.max_seq_length = model.config.max_position_embeddings


EVALUATING: checkpoint-380
==((====))==  Unsloth 2025.6.12: Fast Mistral patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [35]:

# Save the LoRA adapter locally. This saves only the trained weights, not the full model.
adapter_output_dir = "phi3-domain-generator-adapter"
model.save_pretrained(adapter_output_dir)
tokenizer.save_pretrained(adapter_output_dir)

print(f"Adapter saved locally to '{adapter_output_dir}'")


Adapter saved locally to 'phi3-domain-generator-adapter'


In [36]:
drive_adapter_output_dir = "/content/drive/MyDrive/colab_data/tandem/phi3-domain-generator-adapter/phi3-domain-generator-adapter_v2"
model.save_pretrained(drive_adapter_output_dir)
tokenizer.save_pretrained(drive_adapter_output_dir)

print(f"Adapter saved to '{drive_adapter_output_dir}'")

Adapter saved to '/content/drive/MyDrive/colab_data/tandem/phi3-domain-generator-adapter/phi3-domain-generator-adapter_v2'


In [37]:
# --- Cell 7: Inference Setup ---

from peft import PeftModel

# Load the base model and tokenizer again (if in a new session)
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3-mini-4k-instruct-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the LoRA adapter and merge it with the base model
# This combines the original model's knowledge with our fine-tuned specialization.
model = PeftModel.from_pretrained(base_model, "phi3-domain-generator-adapter")

print("Fine-tuned model ready for inference.")


==((====))==  Unsloth 2025.6.12: Fast Mistral patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Fine-tuned model ready for inference.


In [40]:
# --- Cell 8: Inference Function ---

def generate_domains(business_description):
    """Generates domain names using the fine-tuned model."""
    messages = [
        {"role": "user", "content": f"{business_description}"}
    ]

    # Use Unsloth's fast generation pipeline
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

    outputs = model.generate(input_ids=inputs, max_new_tokens=256, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)

    # Extract only the assistant's response
    print(response_text)
    assistant_response_start = response_text[0].find("<|assistant|>")
    if assistant_response_start!= -1:
        return response_text[0][assistant_response_start + len("<|assistant|>"):].strip()
    return "Failed to parse response."


In [41]:
# --- Cell 9: Qualitative Test Cases ---

test_descriptions = [
    "A vintage clothing store that specializes in 90s fashion and streetwear.",
    "A pet grooming service that comes to your house in a mobile van.",
    "An online course platform for learning data science with Python.",
    "A farm-to-table restaurant focusing on sustainable, locally-sourced ingredients.",
    "A financial tech startup that helps millennials invest in cryptocurrency.",
]

for desc in test_descriptions:
    print(f"--- Business Description ---\n{desc}\n")
    generated_output = generate_domains(desc)
    print(f"--- Generated Domains ---\n{generated_output}\n")
    print("="*50)

--- Business Description ---
A vintage clothing store that specializes in 90s fashion and streetwear.

['<|user|> A vintage clothing store that specializes in 90s fashion and streetwear.<|end|><|assistant|> {"domains": ["RetroRock.com", "StreetSense.com", "VintageVibe.com", "NinetiesNest.com", "RetroRevival.com", "FashionFlashback.com", "StreetStyleSphere.com", "RetroRebel.com", "VintageVogue.com", "NinetiesNiche.com"]}<|end|>']
--- Generated Domains ---
{"domains": ["RetroRock.com", "StreetSense.com", "VintageVibe.com", "NinetiesNest.com", "RetroRevival.com", "FashionFlashback.com", "StreetStyleSphere.com", "RetroRebel.com", "VintageVogue.com", "NinetiesNiche.com"]}<|end|>

--- Business Description ---
A pet grooming service that comes to your house in a mobile van.

['<|user|> A pet grooming service that comes to your house in a mobile van.<|end|><|assistant|> {"domains": ["PetMobileGroom.com", "HomePetSpa.com", "VanPaws.com", "GroomVan.com", "PetVanGroom.com", "MobilePetSpa.com", "P